In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import numpy as np
import scipy.integrate as sci
import time 
import threading
import sys
import multiprocessing as multiproc
import datetime
# the function library
from projectlib import *

## Parameters

**Fehler der Parameter wird aus der Datei "(bessere) Optimierung" entnommen**

In [3]:
alpha = 0.18
beta = 0.08
d_alpha = 0.1
d_beta = 0.04

## Initial Values

In [7]:
M =  region_setup(38)[0]
N = import_rki_data(M, 7)[2]
#N_ enthält Anzahl der Bevölkerung der Landkreis
RKI_data = np.load("Internal Data/timeline.npy")
I_RKI = np.array([RKI_data[i][1] for i in range(38)])
#I_RKI[Landkreis][Tag]

In [5]:
dimension = 38

## Jacobian

In [12]:
def I_eff(i, t):
    return effective_infected(comTo, comFrom, N, i, infected, dimension)

def N_eff(i, t):
    return 0

def P(i,j,t):
    return 0


In [14]:
def delta_kronecker(i, j):
    if i == j:
        return 1
    else:
        return 0

def ds(state, number_of_Lks, method, params, i, t):
    """
    calculates dS/dt im Landkreis i zum Zeitpunkt t
    method ist Vektor mit zwei Einträgen 0 ist SIGMA1_1 ist SIGMA_2
    """
    alpha = params[0]
    beta = params[1]
    S_i = state[i][t]
    I_i = state[i + number_of_Lks][t]
    res = 0
    for j in range(number_of_Lks):
        res = res + P(i,j,t) * I_eff(j, t)
    return -method[0] * alpha * S_i * I_i - method[1] * alpha *\
           (N_eff(i)* S_i *I_eff(i, t)  + S_i * res) / N[i]

def dI(state, number_of_Lks, method, params, i, t):
    alpha = params[0]
    beta = params[1] 
    I_i = state[i+number_of_Lks][t]
    res = 0
    for j in range(number_of_Lks):
        res = res + P(i,j,t) * I_eff(j, t)
    return - method[0] * alpha * S_i * I_i + method[1] * alpha *\
           (N_eff(i)* S_i *I_eff(i, t)  + S_i * res) / N[i] - beta * I_i

def dR(state , number_of_Lks , params, i, t):
    beta = params[1]
    p = params[2]
    I_i = state[i + number_of_Lks][t]
    return (1-p) * beta * I_i

def dD(state , number_of_Lks , params, i, t):
    p = params[2]
    I_i = state[i + number_of_Lks][t]
    return (p) * beta * I_i


In [24]:
def Jacobian_SS(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    S = state[0:n-1]
    #I = state[n:2*n-1]
    #R = state[2*n:3*n-1]
    J_SS = np.identity(n)
    for i in range(n):
        J_SS[i][i] = ds(state, number_of_Lks, method, params, i, t)/S[i][t]
    return J_SS

def Jacobian_SI(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    alpha = params[0]
    beta = params[1]
    J_SI = np.identity(n)
    S = state[0:n - 1]
    for i in range(n):
        for l in range(n):
            res1 = -alpha * delta_kronecker(i, l) * method[0] * S[i][t]
            sum1=0
            for j in range(n):
                sum1 = sum1 + P(i,j,t)
            res2 = - alpha * method[1] * N_eff(i) / N[i] * S[i][t] *\
                   (delta_kronecker(i, l) + P(l,i,t)/N[i] - delta_kronecker(i, l) * sum1 / N[i])
            res2 = res2 - alpha * method[1] / N[i] * S[i][t] * P(i,l,t)
            sum2 = 0
            sum3 = 0
            for j in range(n):
                sum2 = sum2 + P(l,j,t) / N[j]
                sum3 = sum3 - P(l,j,t) / N[l] * P(i,l,t)
            res3 = - alpha * method[1] / N[i] * S[i][t] * (sum2 + sum3)
            J_SI[i][l] = res3 + res2 + res1
    return J_SI

def Jacobian_RI(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    alpha = params[0]
    beta = params[1] 
    p = params[2]
    J_RI = np.identity(n)
    for i in range(n):
        for l in range(n):
            J_RI[i][l] = delta_kronecker(i, l) * (1 - p) * beta
    return J_RI

def Jacobian_DI(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    alpha = params[0]
    beta = params[1]
    J_DI = np.identity(n)
    for i in range(n):
        for l in range(n):
            J_DI[i][l] = delta_kronecker(i, l) * p * beta
    return J_DI

def Jacobian_IS(state, number_of_Lks, method, params, t):
    return -1* Jacobian_SI(state, number_of_Lks, method, params, t)

def Jacobian_II(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    alpha = params[0]
    beta = params[1]
    J_II = np.identity(n)
    S = state[0:n - 1]
    for i in range(n):
        for l in range(n):
            sum1 = 0
            sum2 = 0
            sum3 = 0
            for j in range(n):
                sum1 = sum1 - P(i, j, t) / N[i] * delta_kronecker(i, l)
                sum2 = sum2 + P(l, j, t) / N[j]
                sum3 = sum3 - P(l, j, t) / N[l] * P(i, l, t)
            res0 = N_eff(i, t) * (delta_kronecker(i, l) + P(l, i, t) / N[i] + sum1)
            res1 = method[0] * delta_kronecker(i, l) + method[1] / N[i] * res0
            res2 = res1 +  method[1] / N[i] * (P(i,l,t) + sum2 + sum3)
            J_II[i][l] = - beta * delta_kronecker(i, l) + alpha* S[i] * res2
    return J_II

In [26]:
def Jacobian(state, number_of_Lks, method, params, t):
    J_SS = Jacobian_SS(state, number_of_Lks, method, params,t)
    J_II = Jacobian_II(state, number_of_Lks, method, params, t)
    J_RI = Jacobian_RI(state, number_of_Lks, method, params, t)
    J_IS = Jacobian_IS(state, number_of_Lks, method, params, t)
    J_SI = Jacobian_SI(state, number_of_Lks, method, params, t)
    J_DI = Jacobian_DI(state, number_of_Lks, method, params, t)
    J_SR = np.identity(number_of_Lks)*0
    J_SD = np.identity(number_of_Lks) * 0
    J_IR = np.identity(number_of_Lks)*0
    J_ID = np.identity(number_of_Lks)*0
    J_RS = np.identity(number_of_Lks)*0
    J_DS = np.identity(number_of_Lks)*0
    J_RR = np.identity(number_of_Lks)*0
    J_DR = np.identity(number_of_Lks)*0
    J_RD = np.identity(number_of_Lks)*0
    J_DD = np.identity(number_of_Lks)*0
    Z = [J_SS, J_SI, J_SR, J_SD]
    n = number_of_Lks
    J = np.identity(n*4)
    for i in range(n):
        for j in range(n):
            J[i][j]=J_SS[i][j]
            J[i+n][j]= J_SI[i][j]
            J[i + 2*n][j] = J_SR[i][j]
            J[i + 3 * n][j] = J_SD[i][j]
            J[i][j+ n] = J_IS[i][j]
            J[i][j + 2* n] = J_RS[i][j]
            J[i][j + 3 * n] = J_DS[i][j]
            J[i + n][j + n] = J_II[i][j]
            J[i + n][j + 2 * n] = J_RI[i][j]
            J[i + n][j + 3 * n] = J_DI[i][j]
            J[i + 2 * n][j + n] = J_IR[i][j]
            J[i + 2 * n][j + 2*n] = J_RR[i][j]
            J[i + 2 * n][j + 3 * n] = J_DR[i][j]
            J[i + 3 * n][j +  n] = J_ID[i][j]
            J[i + 3 * n][j + 2*n] = J_RD[i][j]
            J[i + 3 * n][j + 3 * n] = J_DD[i][j]
    return J



In [35]:
def S_alpha(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    S = state[0:n-1]
    I = state[n:2*n-1]
    #R = state[2*n:3*n-1]
    S_a = np.array([0 for i in range(n)])
    for i in range(n):
        res0 = - method[0] * S[i][t] * I[i][t]
        res1 = - method[1] * N_eff(i, t) * S[i][t] * I[i][t] / N[i]
        sum0 = 0
        for j in range(n):
            sum0 = sum0 + P(i, j) / n[i] * I_eff(j, t)
        res2 = - method[1] * S[i][t] * sum0
        S_a[i] = res0 + res1 + res2
    return S_a

def S_beta(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    return np.array([0 for i in range(n)])

def S_p(state, number_of_Lks, method, params, t):
    return S_beta(state, number_of_Lks, method, params, t)

def I_alpha(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    S = state[0:n-1]
    I = state[n:2*n-1]
    #R = state[2*n:3*n-1]
    I_a = np.array([0 for i in range(n)])
    for i in range(n):
        res0 = method[0] * S[i][t] * I[i][t]
        res1 = method[1] * I_eff(i, t) * N_eff(i, t) * S[i][t] / N[i]
        sum0 = 0
        for j in range(n):
            sum0 = sum0 + P(i,j,t) *I_eff(j, t) / n[i]
        res2 =  sum0 * S[i][t] * method[1]
        I_a[i] = res0 + res1 + res2  
    return I_a

def I_beta(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    I = state[n:2*n-1]
    return - I

def I_p(state, number_of_Lks, method, params, t):
    return S_beta(state, number_of_Lks, method, params, t)

def R_alpha(state, number_of_Lks, method, params, t):
    return S_beta(state, number_of_Lks, method, params, t)

def R_beta(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    S = state[0:n-1]
    I = state[n:2*n-1]
    #R = state[2*n:3*n-1]
    p = params[2]
    R_b = np.array([(1 - p) * I[i][t] for i in range(n)])
    return R_b

def R_p(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    I = state[n:2*n-1]
    beta = params[1]
    R = np.array([- beta * I[i][t] for i in range(n)])
    return R

def D_alpha(state, number_of_Lks, method, params, t):
    return S_beta(state, number_of_Lks, method, params, t)

def D_beta(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    I = state[n:2*n-1]
    beta = params[1]
    p = params[2]
    D_b = np.array([p * I[i][t] for i in range(n)])
    return D_b

def D_p(state, number_of_Lks, method, params, t):
    n = number_of_Lks
    I = state[n:2*n-1]
    beta = params[1]
    p = params[2]
    Dp = np.array([beta * I[i][t] for i in range(n)])
    return Dp
    

In [39]:
def Jacobian_F(state, number_of_Lks, method, params, t):
    
    Dp = D_p(state, number_of_Lks, method, params, t)
    D_b = D_beta(state, number_of_Lks, method, params, t)
    D_a = D_alpha(state, number_of_Lks, method, params, t)
    Rp = R_p(state, number_of_Lks, method, params, t)
    R_b = R_beta(state, number_of_Lks, method, params, t)
    R_a = R_alpha(state, number_of_Lks, method, params, t)
    Ip = I_p(state, number_of_Lks, method, params, t)
    I_b = I_beta(state, number_of_Lks, method, params, t)
    I_a = I_alpha(state, number_of_Lks, method, params, t)
    Sp = S_p(state, number_of_Lks, method, params, t)
    S_b = S_beta(state, number_of_Lks, method, params, t)
    S_a = S_alpha(state, number_of_Lks, method, params, t)
    S = np.concatenate((S_a, S_b, Sp), axis=0, out=None, dtype=None, casting="same_kind")
    I = np.concatenate((I_a, I_b, Ip), axis=0, out=None, dtype=None, casting="same_kind")
    R = np.concatenate((R_a, R_b, Rp), axis=0, out=None, dtype=None, casting="same_kind")
    D = np.concatenate((D_a, D_b, Dp), axis=0, out=None, dtype=None, casting="same_kind")
    return np.array(S,R,I,D)

## Fehler propagator

In [29]:
def u_neu(u_j, d_p, dt, params, state, number_of_Lks, method):
    n = number_of_Lks
    mat1 = np.identity(n*4) + dt * Jacobian(state, number_of_Lks, method, params, t)
    mat2 = dt * Jacobian_F(state, number_of_Lks, method, params, t)
    return mat1.dot(u_j) + mat2.dot(d_p)

def propagator(state, solt, number_of_Lks, method, params, d_p, u_0):
    u = []
    u_j = u_0
    u.append(u_j)
    for j in range(len(solt)-1):
        dt = solt[t+1] - solt[t]
        u_j = u_neu(u_j, d_p, dt, params, state, number_of_Lks, method)
        u.append(u_j)
    return np.array(u)
    

### To do
 - **define/find N_eff, I_eff**
 - **import P**
 - **test for bugs**
 - **Jacobian_F documentation**
 - **Plots**
 
